# RGBT TIFF Creation from RGB + Thermal Images

This notebook aligns RGB and thermal images and creates 4-channel TIFF files (R,G,B,T) for further processing or training. The alignment parameters were determined manually for this dataset and may need to be adjusted for other cameras or acquisition setups.

In [ ]:
import cv2
import numpy as np
import tifffile
import os
import glob

INPUT_DIR = "input"
OUTPUT_DIR = "output"

# ALIGNMENT PARAMETERS (MANUALLY DETERMINED)
W_THERM_ORIG, H_THERM_ORIG = 1280, 1024
W_RGB_ORIG, H_RGB_ORIG = 3664, 2748
W_THERM_RESIZED = 3621.0
H_THERM_RESIZED = 2897.0
SHIFT_X = 27.0
SHIFT_Y = -74.0
SCALE_X = W_THERM_RESIZED / W_THERM_ORIG
SCALE_Y = H_THERM_RESIZED / H_THERM_ORIG

Run the following cell:

In [ ]:
M_THERM_TO_RGB = np.float32([
    [SCALE_X, 0, SHIFT_X],
    [0, SCALE_Y, SHIFT_Y]
])

# MAIN PROCESSING FUNCTION
def process_folder():
    if not os.path.exists(OUTPUT_DIR):
        os.makedirs(OUTPUT_DIR)

    # Search for RGB images with case-insensitive extensions (.jpg, .JPG, .jpeg)
    types = ('*_Z.jpg', '*_Z.JPG', '*_W.jpg', '*_W.JPG')
    rgb_files = []
    for files in types:
        rgb_files.extend(glob.glob(os.path.join(INPUT_DIR, files)))
    rgb_files = sorted(list(set(rgb_files)))
    print(f"Found {len(rgb_files)} potential RGB images.")

    count = 0
    for rgb_path in rgb_files:
        filename = os.path.basename(rgb_path)
        name_root, ext = os.path.splitext(filename)
        if name_root.endswith("_Z"):
            therm_name_root = name_root[:-2] + "_T"
        elif name_root.endswith("_W"):
            therm_name_root = name_root[:-2] + "_T"
        else:
            continue
        therm_filename = therm_name_root + ext
        therm_path = os.path.join(INPUT_DIR, therm_filename)
        if not os.path.exists(therm_path):
            if ext == ".jpg":
                alt_therm = therm_name_root + ".JPG"
            else:
                alt_therm = therm_name_root + ".jpg"
            alt_path = os.path.join(INPUT_DIR, alt_therm)
            if os.path.exists(alt_path):
                therm_path = alt_path
                therm_filename = alt_therm
            else:
                print(f"Pair not found")
                print(f"   RGB:     {filename}")
                print(f"   EXPECTED: {therm_filename}")
                print("-" * 30)
                continue
        print(f"Processing: {filename} + {therm_filename}")
        try:
            img_rgb = cv2.imread(rgb_path)
            img_therm = cv2.imread(therm_path, cv2.IMREAD_GRAYSCALE)
            if img_rgb is None or img_therm is None:
                print("Could not read files")
                continue
            img_rgb_aligned = cv2.warpAffine(
                img_rgb,
                M_THERM_TO_RGB,
                (W_THERM_ORIG, H_THERM_ORIG),
                flags=cv2.INTER_LANCZOS4 | cv2.WARP_INVERSE_MAP,
                borderMode=cv2.BORDER_CONSTANT,
                borderValue=(0, 0, 0)
            )
            b, g, r = cv2.split(img_rgb_aligned)
            merged_hwc = cv2.merge([r, g, b, img_therm])
            merged_chw = np.transpose(merged_hwc, (2, 0, 1))
            output_filename = filename.replace(ext, "_RGBT.tiff")
            output_filename = output_filename.replace("_Z_", "_").replace("_W_", "_")
            output_path = os.path.join(OUTPUT_DIR, output_filename)
            tifffile.imwrite(output_path, merged_chw)
            count += 1
        except Exception as e:
            print(f"Error on {filename}: {e}")
    print(f"\nCreated {count} TIFF images.")

In [ ]:
process_folder()